# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV


from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')

df = pd.read_sql("SELECT * FROM InsertTableName", engine)


In [3]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,...,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000
mean,15224.871333,0.766470,0.170666,0.004501,0.414267,0.079496,0.050086,0.027618,0.017967,0.032806,...,0.011787,0.043906,0.278352,0.082205,0.093191,0.010757,0.093649,0.020217,0.052489,0.193591
std,8827.053788,0.423085,0.376224,0.066941,0.492604,0.270517,0.218126,0.163878,0.132833,0.178131,...,0.107929,0.204890,0.448196,0.274682,0.290705,0.103160,0.291345,0.140746,0.223015,0.395120
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15663.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
X = df['message']

# categories are all columns after the 4th column
y = df.iloc[:,4:]

### 2. Tokenization function to process  text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build Machine learning pipeline
This machine learning pipeline takes in the `message` column as input, and outputs the classification results on the other 36 categories in the dataset using the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) classifier for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test  model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
# overall accuracy

accuracy = (y_pred == y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# classification report on test data

print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

Accuracy 94.11% 

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5024
               request       0.81      0.39      0.52      1080
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.54      0.63      2702
          medical_help       0.54      0.05      0.10       509
      medical_products       0.64      0.09      0.16       303
     search_and_rescue       0.62      0.08      0.15       179
              security       0.00      0.00      0.00       120
              military       0.57      0.06      0.11       221
                 water       0.84      0.16      0.27       438
                  food       0.84      0.34      0.48       699
               shelter       0.82      0.16      0.27       599
              clothing       0.80      0.12      0.21        99
                 money       0.80      0.02      0.05       165
        missing_peopl

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# test msg

msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related' 'weather_related']


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
# check pipeline parameters
# pipeline.get_params()

parameters = {
#         'clf__estimator': ['RandomForestClassifier()','ExtraTreesClassifier()'],
        'clf__estimator__n_estimators': [10, 50]
    } 

cv = GridSearchCV(estimator=pipeline, n_jobs = -1, param_grid=parameters)

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

Try:
Inherently multiclass: Naive Bayes, LDA and QDA, Decision Trees, Random Forests, Nearest Neighbors, setting multi_class='multinomial' in sklearn.linear_model.LogisticRegression.
Support multilabel: Decision Trees, Random Forests, Nearest Neighbors.
One-Vs-One: sklearn.svm.SVC.
One-Vs-All: all linear models except sklearn.svm.SVC.

In [14]:
y_pred = cv.predict(X_test)

In [15]:
# overall accuracy
accuracy = (y_pred == y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# classification report on test data
print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

print("\nBest Parameters:", cv.best_params_)


Accuracy 94.49% 

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5024
               request       0.89      0.42      0.57      1080
                 offer       0.00      0.00      0.00        28
           aid_related       0.79      0.59      0.68      2702
          medical_help       0.67      0.07      0.13       509
      medical_products       0.68      0.06      0.10       303
     search_and_rescue       0.50      0.02      0.03       179
              security       0.50      0.01      0.02       120
              military       0.64      0.03      0.06       221
                 water       0.89      0.21      0.34       438
                  food       0.86      0.43      0.57       699
               shelter       0.88      0.21      0.34       599
              clothing       0.78      0.07      0.13        99
                 money       0.75      0.02      0.04       165
        missing_peopl

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
# Create custom transformer

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [17]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [18]:
parameters = {
#         'clf__estimator__learning_rate': [0.1, 0.5, 1],
        'clf__estimator__n_estimators': [10, 50]

    } 
cv = GridSearchCV(estimator=pipeline, n_jobs = -1, param_grid=parameters)

In [19]:
# train pipeline
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [20]:
# overall accuracy
accuracy = (y_pred == y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# classification report on test data
print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

print("\nBest Parameters:", cv.best_params_)

Accuracy 94.49% 

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5024
               request       0.87      0.41      0.56      1080
                 offer       0.00      0.00      0.00        28
           aid_related       0.80      0.60      0.69      2702
          medical_help       0.63      0.06      0.12       509
      medical_products       0.73      0.07      0.13       303
     search_and_rescue       0.67      0.02      0.04       179
              security       0.33      0.01      0.02       120
              military       0.77      0.05      0.09       221
                 water       0.92      0.21      0.35       438
                  food       0.84      0.43      0.57       699
               shelter       0.86      0.27      0.41       599
              clothing       0.89      0.08      0.15        99
                 money       0.67      0.01      0.02       165
        missing_peopl

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [21]:
m = pickle.dumps('classifier.pkl')